# Concrete Mixture Machine Learning Analysis
## Prediction of Cost, Slump, Compressive Strength, and CO2 Emissions
### Analysis across 7 different ash types

## 1. SETUP AND IMPORTS

In [ ]:
# Install required packages
!pip install -q openpyxl scikit-learn xgboost lightgbm catboost matplotlib seaborn pandas numpy joblib

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os
from scipy import stats

# Machine Learning imports
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("All libraries imported successfully!")

## 2. DATA LOADING

In [ ]:
# Load the dataset
excel_file = 'REVISED_DATASET.xlsx'

# Read all sheets (first 7 are ash types)
xl = pd.ExcelFile(excel_file)
ash_types = xl.sheet_names[:7]  # First 7 sheets are ash types

print("Ash Types Found:")
for i, ash in enumerate(ash_types, 1):
    print(f"{i}. {ash}")

# Load all datasets
datasets_raw = {}
for ash_type in ash_types:
    datasets_raw[ash_type] = pd.read_excel(excel_file, sheet_name=ash_type)
    print(f"\n{ash_type}: {datasets_raw[ash_type].shape[0]} rows, {datasets_raw[ash_type].shape[1]} columns")

## 3. DATA EXPLORATION (BEFORE CLEANING)

In [ ]:
# Show histogram of entries before cleaning
entries_before = {ash: len(datasets_raw[ash]) for ash in ash_types}

plt.figure(figsize=(12, 6))
plt.bar(range(len(entries_before)), list(entries_before.values()), color='steelblue', alpha=0.7)
plt.xlabel('Ash Type', fontsize=12, fontweight='bold')
plt.ylabel('Number of Entries', fontsize=12, fontweight='bold')
plt.title('Number of Entries per Ash Type (Before Cleaning)', fontsize=14, fontweight='bold')
plt.xticks(range(len(entries_before)), [ash.replace(' 1', '') for ash in entries_before.keys()], rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nEntries before cleaning:")
for ash, count in entries_before.items():
    print(f"{ash}: {count} entries")

In [ ]:
# Display basic info for each ash type
for ash_type in ash_types:
    print(f"\n{'='*60}")
    print(f"ASH TYPE: {ash_type}")
    print(f"{'='*60}")
    print(f"\nShape: {datasets_raw[ash_type].shape}")
    print(f"\nColumn Names:")
    print(datasets_raw[ash_type].columns.tolist())
    print(f"\nMissing Values:")
    print(datasets_raw[ash_type].isnull().sum())
    print(f"\nData Types:")
    print(datasets_raw[ash_type].dtypes)

## 4. DATA CLEANING AND PREPARATION

In [ ]:
def clean_dataset(df, ash_type_name):
    """
    Clean and prepare dataset for analysis
    """
    print(f"\nCleaning {ash_type_name}...")
    print(f"Original shape: {df.shape}")
    
    # Create a copy
    df_clean = df.copy()
    
    # Handle CO2 column - convert to numeric
    if 'CO2_kgCO₂e / kg' in df_clean.columns:
        df_clean['CO2_kgCO₂e / kg'] = pd.to_numeric(df_clean['CO2_kgCO₂e / kg'], errors='coerce')
    
    # Define target variables
    target_vars = ['cost_USD_per_m3', 'Slump(mm)', 'compressive_strength_MPa_', 'CO2_kgCO₂e / kg']
    
    # Define input variables
    input_vars = ['replacement_pct', 'cement_kg_m3', 'ash_kg_m3', 'fine_aggregate_kg_m3', 
                  'coarse_aggregate_kg_m3', 'pozzolan added(Fly Ash) kgm3', 
                  'superplasticizer_kg_m3', 'water kg_m3', 'curing_days']
    
    # Keep only necessary columns
    necessary_cols = input_vars + target_vars + ['paper', 'reference']
    df_clean = df_clean[[col for col in necessary_cols if col in df_clean.columns]]
    
    # Remove rows where ALL target variables are missing
    target_cols_present = [col for col in target_vars if col in df_clean.columns]
    df_clean = df_clean.dropna(subset=target_cols_present, how='all')
    
    # Fill missing values in input variables with median
    for col in input_vars:
        if col in df_clean.columns:
            if df_clean[col].isnull().any():
                median_val = df_clean[col].median()
                df_clean[col].fillna(median_val, inplace=True)
                print(f"  Filled {col} missing values with median: {median_val:.2f}")
    
    # Remove duplicates
    before_dup = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=input_vars + target_cols_present)
    after_dup = len(df_clean)
    if before_dup != after_dup:
        print(f"  Removed {before_dup - after_dup} duplicate rows")
    
    # Remove outliers using IQR method for each variable
    before_outliers = len(df_clean)
    for col in input_vars + target_cols_present:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR  # Using 3*IQR for more lenient outlier removal
            upper_bound = Q3 + 3 * IQR
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    after_outliers = len(df_clean)
    if before_outliers != after_outliers:
        print(f"  Removed {before_outliers - after_outliers} outlier rows")
    
    print(f"Final shape: {df_clean.shape}")
    print(f"Rows removed: {df.shape[0] - df_clean.shape[0]} ({100*(df.shape[0] - df_clean.shape[0])/df.shape[0]:.1f}%)")
    
    return df_clean

# Clean all datasets
datasets_clean = {}
for ash_type in ash_types:
    datasets_clean[ash_type] = clean_dataset(datasets_raw[ash_type], ash_type)

In [ ]:
# Show histogram of entries after cleaning
entries_after = {ash: len(datasets_clean[ash]) for ash in ash_types}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Before cleaning
ax1.bar(range(len(entries_before)), list(entries_before.values()), color='steelblue', alpha=0.7)
ax1.set_xlabel('Ash Type', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Entries', fontsize=12, fontweight='bold')
ax1.set_title('Before Cleaning', fontsize=14, fontweight='bold')
ax1.set_xticks(range(len(entries_before)))
ax1.set_xticklabels([ash.replace(' 1', '') for ash in entries_before.keys()], rotation=45, ha='right')
ax1.grid(axis='y', alpha=0.3)

# After cleaning
ax2.bar(range(len(entries_after)), list(entries_after.values()), color='coral', alpha=0.7)
ax2.set_xlabel('Ash Type', fontsize=12, fontweight='bold')
ax2.set_ylabel('Number of Entries', fontsize=12, fontweight='bold')
ax2.set_title('After Cleaning', fontsize=14, fontweight='bold')
ax2.set_xticks(range(len(entries_after)))
ax2.set_xticklabels([ash.replace(' 1', '') for ash in entries_after.keys()], rotation=45, ha='right')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nComparison of entries:")
print(f"{'Ash Type':<15} {'Before':<10} {'After':<10} {'Removed':<10} {'% Removed'}")
print("-" * 60)
for ash in ash_types:
    before = entries_before[ash]
    after = entries_after[ash]
    removed = before - after
    pct = 100 * removed / before if before > 0 else 0
    print(f"{ash.replace(' 1', ''):<15} {before:<10} {after:<10} {removed:<10} {pct:.1f}%")

## 5. EXPLORATORY DATA ANALYSIS (EDA)

### 5.1 Outlier Visualization for Each Variable by Ash Type

In [ ]:
# Define variables
target_vars = ['cost_USD_per_m3', 'Slump(mm)', 'compressive_strength_MPa_', 'CO2_kgCO₂e / kg']
input_vars = ['replacement_pct', 'cement_kg_m3', 'ash_kg_m3', 'fine_aggregate_kg_m3', 
              'coarse_aggregate_kg_m3', 'pozzolan added(Fly Ash) kgm3', 
              'superplasticizer_kg_m3', 'water kg_m3', 'curing_days']

all_vars = target_vars + input_vars

In [ ]:
# Visualize outliers for TARGET VARIABLES by ash type
print("Creating outlier visualizations for TARGET VARIABLES...\n")

for target_var in target_vars:
    print(f"\nProcessing: {target_var}")
    
    for ash_type in ash_types:
        df = datasets_clean[ash_type]
        
        if target_var in df.columns and not df[target_var].isnull().all():
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Box plot
            axes[0].boxplot(df[target_var].dropna(), vert=True)
            axes[0].set_ylabel(target_var, fontsize=11, fontweight='bold')
            axes[0].set_title(f'Box Plot', fontsize=12, fontweight='bold')
            axes[0].grid(axis='y', alpha=0.3)
            
            # Histogram with KDE
            axes[1].hist(df[target_var].dropna(), bins=30, alpha=0.7, color='steelblue', edgecolor='black')
            axes[1].set_xlabel(target_var, fontsize=11, fontweight='bold')
            axes[1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
            axes[1].set_title(f'Distribution', fontsize=12, fontweight='bold')
            axes[1].grid(axis='y', alpha=0.3)
            
            # Add secondary axis for KDE
            ax2 = axes[1].twinx()
            df[target_var].dropna().plot(kind='kde', ax=ax2, color='red', linewidth=2)
            ax2.set_ylabel('Density', fontsize=11, fontweight='bold')
            ax2.grid(False)
            
            fig.suptitle(f'{target_var} - {ash_type}', fontsize=14, fontweight='bold', y=1.02)
            plt.tight_layout()
            plt.show()

In [ ]:
# Visualize outliers for INPUT VARIABLES by ash type
print("Creating outlier visualizations for INPUT VARIABLES...\n")

for input_var in input_vars:
    print(f"\nProcessing: {input_var}")
    
    for ash_type in ash_types:
        df = datasets_clean[ash_type]
        
        if input_var in df.columns and not df[input_var].isnull().all():
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Box plot
            axes[0].boxplot(df[input_var].dropna(), vert=True)
            axes[0].set_ylabel(input_var, fontsize=11, fontweight='bold')
            axes[0].set_title(f'Box Plot', fontsize=12, fontweight='bold')
            axes[0].grid(axis='y', alpha=0.3)
            
            # Histogram with KDE
            axes[1].hist(df[input_var].dropna(), bins=30, alpha=0.7, color='coral', edgecolor='black')
            axes[1].set_xlabel(input_var, fontsize=11, fontweight='bold')
            axes[1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
            axes[1].set_title(f'Distribution', fontsize=12, fontweight='bold')
            axes[1].grid(axis='y', alpha=0.3)
            
            # Add secondary axis for KDE
            ax2 = axes[1].twinx()
            df[input_var].dropna().plot(kind='kde', ax=ax2, color='darkred', linewidth=2)
            ax2.set_ylabel('Density', fontsize=11, fontweight='bold')
            ax2.grid(False)
            
            fig.suptitle(f'{input_var} - {ash_type}', fontsize=14, fontweight='bold', y=1.02)
            plt.tight_layout()
            plt.show()

### 5.2 Pearson Correlation Matrices

In [ ]:
# Plot Pearson correlation matrix for each ash type
print("Creating correlation matrices for each ash type...\n")

for ash_type in ash_types:
    df = datasets_clean[ash_type]
    
    # Select only numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Calculate correlation matrix
    corr_matrix = df[numeric_cols].corr(method='pearson')
    
    # Create figure
    plt.figure(figsize=(14, 12))
    
    # Create heatmap
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
                vmin=-1, vmax=1)
    
    plt.title(f'Pearson Correlation Matrix - {ash_type}', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    print(f"Correlation matrix created for {ash_type}")
    print("-" * 60)

### 5.3 Frequency Distributions by Ash Type

In [ ]:
# Plot frequency distributions for each variable by ash type
print("Creating frequency distribution plots...\n")

# For each variable, create separate plots for each ash type
for var in all_vars:
    print(f"\nProcessing variable: {var}")
    
    for ash_type in ash_types:
        df = datasets_clean[ash_type]
        
        if var in df.columns and not df[var].isnull().all():
            plt.figure(figsize=(10, 6))
            
            # Create histogram
            n, bins, patches = plt.hist(df[var].dropna(), bins=25, alpha=0.7, 
                                        color='mediumseagreen', edgecolor='black', linewidth=1.2)
            
            # Add KDE overlay
            ax2 = plt.gca().twinx()
            df[var].dropna().plot(kind='kde', ax=ax2, color='darkblue', linewidth=2.5)
            ax2.set_ylabel('Density', fontsize=12, fontweight='bold')
            ax2.grid(False)
            
            plt.gca().set_xlabel(var, fontsize=12, fontweight='bold')
            plt.gca().set_ylabel('Frequency', fontsize=12, fontweight='bold')
            plt.gca().set_title(f'Frequency Distribution of {var}\n{ash_type}', 
                               fontsize=14, fontweight='bold', pad=15)
            plt.gca().grid(axis='y', alpha=0.3)
            
            # Add statistics box
            stats_text = f"Mean: {df[var].mean():.2f}\nMedian: {df[var].median():.2f}\nStd: {df[var].std():.2f}"
            plt.gca().text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
                          fontsize=10, verticalalignment='top',
                          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            plt.tight_layout()
            plt.show()

## 6. MODEL TRAINING PREPARATION

In [ ]:
# Define the 10 machine learning models
def get_models():
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'ElasticNet': ElasticNet(),
        'Decision Tree': DecisionTreeRegressor(random_state=42),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'XGBoost': XGBRegressor(random_state=42, verbosity=0),
        'LightGBM': LGBMRegressor(random_state=42, verbose=-1),
        'CatBoost': CatBoostRegressor(random_state=42, verbose=0)
    }
    return models

# Define hyperparameter grids for tuning
def get_param_grids():
    param_grids = {
        'Linear Regression': {},
        'Ridge Regression': {'alpha': [0.1, 1.0, 10.0]},
        'Lasso Regression': {'alpha': [0.1, 1.0, 10.0]},
        'ElasticNet': {'alpha': [0.1, 1.0], 'l1_ratio': [0.3, 0.5, 0.7]},
        'Decision Tree': {'max_depth': [5, 10, 15], 'min_samples_split': [2, 5]},
        'Random Forest': {'n_estimators': [50, 100], 'max_depth': [10, 20], 'min_samples_split': [2, 5]},
        'Gradient Boosting': {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]},
        'XGBoost': {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]},
        'LightGBM': {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'num_leaves': [31, 50]},
        'CatBoost': {'iterations': [50, 100], 'learning_rate': [0.01, 0.1], 'depth': [4, 6]}
    }
    return param_grids

print("Model definitions ready!")
print(f"\nModels to be trained: {len(get_models())}")
for i, model_name in enumerate(get_models().keys(), 1):
    print(f"{i}. {model_name}")

In [ ]:
# Function to split data into train (75%), test (15%), and validation (10%)
def split_data(X, y, random_state=42):
    # First split: 75% train, 25% temp
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=random_state)
    
    # Second split: 15% test (60% of temp), 10% validation (40% of temp)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.4, random_state=random_state)
    
    return X_train, X_test, X_val, y_train, y_test, y_val

print("Data splitting function ready!")
print("Split ratios: Train=75%, Test=15%, Validation=10%")

## 7. MODEL TRAINING AND EVALUATION

In [ ]:
# Initialize storage for results
all_results = {}
best_models = {}

# Create directory for saving models
os.makedirs('trained_models', exist_ok=True)

print("Starting model training across all ash types and target variables...")
print("=" * 80)

In [ ]:
# Train models for each ash type and target variable
for ash_type in ash_types:
    print(f"\n\n{'#'*80}")
    print(f"# PROCESSING ASH TYPE: {ash_type}")
    print(f"{'#'*80}\n")
    
    df = datasets_clean[ash_type]
    
    # Initialize results storage for this ash type
    all_results[ash_type] = {}
    best_models[ash_type] = {}
    
    # Prepare input features
    X_cols = [col for col in input_vars if col in df.columns]
    
    for target_var in target_vars:
        if target_var not in df.columns or df[target_var].isnull().all():
            print(f"  ⚠ Skipping {target_var} - not available in {ash_type}")
            continue
        
        print(f"\n{'='*80}")
        print(f"TARGET VARIABLE: {target_var}")
        print(f"{'='*80}")
        
        # Prepare data
        df_target = df.dropna(subset=[target_var])
        X = df_target[X_cols]
        y = df_target[target_var]
        
        if len(X) < 10:
            print(f"  ⚠ Insufficient data ({len(X)} samples) for {target_var}")
            continue
        
        print(f"\nDataset size: {len(X)} samples")
        
        # Split data
        X_train, X_test, X_val, y_train, y_test, y_val = split_data(X, y)
        
        print(f"Train: {len(X_train)} | Test: {len(X_test)} | Validation: {len(X_val)}")
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_val_scaled = scaler.transform(X_val)
        
        # Train models
        models = get_models()
        param_grids = get_param_grids()
        
        results = {}
        
        for model_name, model in models.items():
            print(f"\n  Training {model_name}...")
            
            try:
                # Hyperparameter tuning
                if param_grids[model_name]:
                    grid_search = GridSearchCV(model, param_grids[model_name], 
                                              cv=min(3, len(X_train)//10 if len(X_train) >= 30 else 2),
                                              scoring='r2', n_jobs=-1)
                    grid_search.fit(X_train_scaled, y_train)
                    best_model = grid_search.best_estimator_
                    print(f"    Best params: {grid_search.best_params_}")
                else:
                    best_model = model
                    best_model.fit(X_train_scaled, y_train)
                
                # Predictions
                y_train_pred = best_model.predict(X_train_scaled)
                y_test_pred = best_model.predict(X_test_scaled)
                y_val_pred = best_model.predict(X_val_scaled)
                
                # Calculate metrics
                results[model_name] = {
                    'model': best_model,
                    'scaler': scaler,
                    'train_r2': r2_score(y_train, y_train_pred),
                    'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
                    'train_mae': mean_absolute_error(y_train, y_train_pred),
                    'test_r2': r2_score(y_test, y_test_pred),
                    'test_rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
                    'test_mae': mean_absolute_error(y_test, y_test_pred),
                    'val_r2': r2_score(y_val, y_val_pred),
                    'val_rmse': np.sqrt(mean_squared_error(y_val, y_val_pred)),
                    'val_mae': mean_absolute_error(y_val, y_val_pred)
                }
                
                print(f"    Train R²: {results[model_name]['train_r2']:.4f} | Test R²: {results[model_name]['test_r2']:.4f} | Val R²: {results[model_name]['val_r2']:.4f}")
                
            except Exception as e:
                print(f"    ✗ Error: {str(e)}")
                continue
        
        # Store results
        all_results[ash_type][target_var] = results
        
        # Find best model based on validation R²
        if results:
            best_model_name = max(results.keys(), key=lambda x: results[x]['val_r2'])
            best_models[ash_type][target_var] = {
                'name': best_model_name,
                'model': results[best_model_name]['model'],
                'scaler': results[best_model_name]['scaler'],
                'metrics': results[best_model_name],
                'features': X_cols
            }
            
            print(f"\n  ★ BEST MODEL: {best_model_name}")
            print(f"    Validation R²: {results[best_model_name]['val_r2']:.4f}")
            print(f"    Validation RMSE: {results[best_model_name]['val_rmse']:.4f}")
            
            # Save best model
            model_path = f"trained_models/{ash_type.replace(' ', '_')}_{target_var.replace(' ', '_').replace('/', '_')}_best.pkl"
            joblib.dump({
                'model': results[best_model_name]['model'],
                'scaler': results[best_model_name]['scaler'],
                'features': X_cols,
                'ash_type': ash_type,
                'target_var': target_var,
                'model_name': best_model_name
            }, model_path)
            print(f"    Model saved: {model_path}")

print("\n\n" + "="*80)
print("MODEL TRAINING COMPLETED!")
print("="*80)

## 8. MODEL COMPARISON AND VISUALIZATION

In [ ]:
# Create comparison tables for each ash type and target variable
print("Creating model comparison tables...\n")

for ash_type in ash_types:
    if ash_type not in all_results or not all_results[ash_type]:
        continue
        
    print(f"\n{'='*100}")
    print(f"ASH TYPE: {ash_type}")
    print(f"{'='*100}")
    
    for target_var in target_vars:
        if target_var not in all_results[ash_type] or not all_results[ash_type][target_var]:
            continue
        
        results = all_results[ash_type][target_var]
        
        print(f"\nTarget: {target_var}")
        print("-" * 100)
        print(f"{'Model':<25} {'Train R²':<12} {'Test R²':<12} {'Val R²':<12} {'Val RMSE':<12} {'Val MAE':<12}")
        print("-" * 100)
        
        # Sort by validation R²
        sorted_results = sorted(results.items(), key=lambda x: x[1]['val_r2'], reverse=True)
        
        for model_name, metrics in sorted_results:
            print(f"{model_name:<25} {metrics['train_r2']:<12.4f} {metrics['test_r2']:<12.4f} "
                  f"{metrics['val_r2']:<12.4f} {metrics['val_rmse']:<12.4f} {metrics['val_mae']:<12.4f}")
        
        print("-" * 100)
        print(f"★ Best: {sorted_results[0][0]} (Val R²: {sorted_results[0][1]['val_r2']:.4f})")

In [ ]:
# Visualize model performance comparison
print("\nCreating performance comparison visualizations...\n")

for ash_type in ash_types:
    if ash_type not in all_results or not all_results[ash_type]:
        continue
    
    for target_var in target_vars:
        if target_var not in all_results[ash_type] or not all_results[ash_type][target_var]:
            continue
        
        results = all_results[ash_type][target_var]
        
        # Create comparison plot
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        model_names = list(results.keys())
        train_r2 = [results[m]['train_r2'] for m in model_names]
        test_r2 = [results[m]['test_r2'] for m in model_names]
        val_r2 = [results[m]['val_r2'] for m in model_names]
        
        x = np.arange(len(model_names))
        width = 0.25
        
        # R² comparison
        axes[0].bar(x - width, train_r2, width, label='Train', alpha=0.8, color='steelblue')
        axes[0].bar(x, test_r2, width, label='Test', alpha=0.8, color='coral')
        axes[0].bar(x + width, val_r2, width, label='Validation', alpha=0.8, color='mediumseagreen')
        axes[0].set_xlabel('Model', fontsize=11, fontweight='bold')
        axes[0].set_ylabel('R² Score', fontsize=11, fontweight='bold')
        axes[0].set_title('R² Score Comparison', fontsize=12, fontweight='bold')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(model_names, rotation=45, ha='right', fontsize=9)
        axes[0].legend()
        axes[0].grid(axis='y', alpha=0.3)
        
        # RMSE comparison
        val_rmse = [results[m]['val_rmse'] for m in model_names]
        axes[1].bar(model_names, val_rmse, alpha=0.8, color='indianred')
        axes[1].set_xlabel('Model', fontsize=11, fontweight='bold')
        axes[1].set_ylabel('RMSE', fontsize=11, fontweight='bold')
        axes[1].set_title('Validation RMSE', fontsize=12, fontweight='bold')
        axes[1].tick_params(axis='x', rotation=45, labelsize=9)
        plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
        axes[1].grid(axis='y', alpha=0.3)
        
        # MAE comparison
        val_mae = [results[m]['val_mae'] for m in model_names]
        axes[2].bar(model_names, val_mae, alpha=0.8, color='mediumpurple')
        axes[2].set_xlabel('Model', fontsize=11, fontweight='bold')
        axes[2].set_ylabel('MAE', fontsize=11, fontweight='bold')
        axes[2].set_title('Validation MAE', fontsize=12, fontweight='bold')
        axes[2].tick_params(axis='x', rotation=45, labelsize=9)
        plt.setp(axes[2].xaxis.get_majorticklabels(), rotation=45, ha='right')
        axes[2].grid(axis='y', alpha=0.3)
        
        fig.suptitle(f'Model Performance Comparison\n{ash_type} - {target_var}', 
                    fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()

## 9. FEATURE IMPORTANCE ANALYSIS

In [ ]:
# Visualize feature importance for best models
print("Creating feature importance visualizations for best models...\n")

for ash_type in ash_types:
    if ash_type not in best_models or not best_models[ash_type]:
        continue
    
    for target_var in target_vars:
        if target_var not in best_models[ash_type]:
            continue
        
        best_model_info = best_models[ash_type][target_var]
        model = best_model_info['model']
        model_name = best_model_info['name']
        features = best_model_info['features']
        
        # Extract feature importance
        feature_importance = None
        
        if hasattr(model, 'feature_importances_'):
            # Tree-based models
            feature_importance = model.feature_importances_
        elif hasattr(model, 'coef_'):
            # Linear models
            feature_importance = np.abs(model.coef_)
        
        if feature_importance is not None:
            # Create DataFrame for visualization
            importance_df = pd.DataFrame({
                'Feature': features,
                'Importance': feature_importance
            }).sort_values('Importance', ascending=False)
            
            # Normalize to sum to 1 (weights)
            importance_df['Weight'] = importance_df['Importance'] / importance_df['Importance'].sum()
            
            # Create visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
            
            # Bar plot
            colors = plt.cm.viridis(np.linspace(0, 1, len(importance_df)))
            ax1.barh(importance_df['Feature'], importance_df['Weight'], color=colors, alpha=0.8)
            ax1.set_xlabel('Normalized Weight', fontsize=12, fontweight='bold')
            ax1.set_ylabel('Feature', fontsize=12, fontweight='bold')
            ax1.set_title('Feature Weights (Normalized)', fontsize=13, fontweight='bold')
            ax1.grid(axis='x', alpha=0.3)
            
            # Pie chart
            ax2.pie(importance_df['Weight'], labels=importance_df['Feature'], autopct='%1.1f%%',
                   startangle=90, colors=colors)
            ax2.set_title('Feature Weight Distribution', fontsize=13, fontweight='bold')
            
            fig.suptitle(f'Feature Importance - Best Model ({model_name})\n{ash_type} - {target_var}',
                        fontsize=14, fontweight='bold', y=0.98)
            plt.tight_layout()
            plt.show()
            
            print(f"\nFeature weights for {ash_type} - {target_var}:")
            print(importance_df[['Feature', 'Weight']].to_string(index=False))
            print("-" * 60)

## 10. SUMMARY OF BEST MODELS

In [ ]:
# Create summary table of best models
print("\n" + "="*120)
print("SUMMARY OF BEST MODELS PER ASH TYPE AND TARGET VARIABLE")
print("="*120)

summary_data = []

for ash_type in ash_types:
    if ash_type not in best_models or not best_models[ash_type]:
        continue
    
    for target_var in target_vars:
        if target_var not in best_models[ash_type]:
            continue
        
        best_model_info = best_models[ash_type][target_var]
        metrics = best_model_info['metrics']
        
        summary_data.append({
            'Ash Type': ash_type.replace(' 1', ''),
            'Target Variable': target_var,
            'Best Model': best_model_info['name'],
            'Train R²': f"{metrics['train_r2']:.4f}",
            'Test R²': f"{metrics['test_r2']:.4f}",
            'Val R²': f"{metrics['val_r2']:.4f}",
            'Val RMSE': f"{metrics['val_rmse']:.4f}",
            'Val MAE': f"{metrics['val_mae']:.4f}"
        })

summary_df = pd.DataFrame(summary_data)
print("\n", summary_df.to_string(index=False))
print("\n" + "="*120)

# Save summary to CSV
summary_df.to_csv('trained_models/best_models_summary.csv', index=False)
print("\nSummary saved to: trained_models/best_models_summary.csv")

## 11. EXPORT MODELS FOR GUI

In [ ]:
# Create a comprehensive model registry for the GUI
model_registry = {
    'ash_types': ash_types,
    'target_variables': target_vars,
    'input_variables': input_vars,
    'models': best_models
}

# Save registry
joblib.dump(model_registry, 'trained_models/model_registry.pkl')
print("Model registry saved: trained_models/model_registry.pkl")

print("\n" + "="*80)
print("ALL MODELS EXPORTED SUCCESSFULLY!")
print("="*80)
print(f"\nTotal models saved: {sum(len(best_models[ash]) for ash in best_models)}")
print(f"Storage location: trained_models/")
print("\nReady for GUI application!")

## 12. COMPLETION

In [ ]:
print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*20 + "ANALYSIS COMPLETED SUCCESSFULLY!" + " "*25 + "#")
print("#" + " "*78 + "#")
print("#"*80)
print("\nNext Steps:")
print("1. Download the 'trained_models' folder")
print("2. Run the GUI application (concrete_predictor_gui.py)")
print("3. Make predictions using the best models for each target variable")
print("\n" + "#"*80)